In [1]:
import pandas as pd

In [17]:
### Dataset from IMDB with game and genres classified ### 
imdb_df = pd.read_csv('imdb-videogames.csv', index_col=0)
vgs = pd.read_csv('Video_Games_Sales_as_at_22_Dec_2016.csv')


,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
matches = []
for title in vgs.Name.unique():
    if title in imdb_df.name.unique():
        matches.append(title)
len(matches)

3619

In [3]:
imdb_df.drop(axis = 1, labels = ['url', 'plot'], inplace= True)    ### Unecessary columns that wont be using 
imdb_df.replace({True: 1, False: 0}, inplace= True)   ### Less text and made to be simplier
imdb_df.drop(imdb_df[imdb_df.year < 1996].index, inplace = True)   ### Older games not considered in GOTY is excluded from the dataset
imdb_df.drop(imdb_df[imdb_df.year > 2023].index, inplace = True)   ### Newer games not considered in GOTY is excluded from the dataset for now
imdb_df.year.fillna(0, inplace= True)   ### Any year missing is filled with a temperary 0
imdb_df.year = imdb_df.year.apply(lambda x: round(x))   ### Remove the extra .0 in the years   

imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16158 entries, 0 to 20802
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         16158 non-null  object 
 1   year         16158 non-null  int64  
 2   certificate  6995 non-null   object 
 3   rating       9153 non-null   float64
 4   votes        9153 non-null   object 
 5   Action       16158 non-null  int64  
 6   Adventure    16158 non-null  int64  
 7   Comedy       16158 non-null  int64  
 8   Crime        16158 non-null  int64  
 9   Family       16158 non-null  int64  
 10  Fantasy      16158 non-null  int64  
 11  Mystery      16158 non-null  int64  
 12  Sci-Fi       16158 non-null  int64  
 13  Thriller     16158 non-null  int64  
dtypes: float64(1), int64(10), object(3)
memory usage: 1.8+ MB


In [4]:
imdb_df.year.loc[imdb_df.rating.isna()].value_counts()

2019    576
2018    481
2021    441
2017    436
2016    420
2020    405
2015    349
2022    311
2014    294
0       266
2012    264
2013    256
2011    250
2010    203
2009    203
2006    169
2008    167
1996    160
2001    149
2007    148
1997    142
1998    138
2002    126
2005    125
2003    124
2004    123
2000    117
1999    109
2023     53
Name: year, dtype: int64

In [5]:
imdb_df[imdb_df.year > 2023]

,name,year,certificate,rating,votes,Action,Adventure,Comedy,Crime,Family,Fantasy,Mystery,Sci-Fi,Thriller


In [6]:
GOTY_df = pd.read_excel('Sample GOTY.xlsx')
GOTY_df.head()

,Year,Game,Genre,Developer(s)
0,1998,GoldenEye 007,First-person shooter,Rare
1,1999,The Legend of Zelda: Ocarina of Time,Action-adventure,Nintendo EAD
2,2000,MediEvil 2,Action-adventure,SCE Cambridge Studio
3,2000,Pokémon Yellow,Role-playing,Game Freak
4,2000,Deus Ex,Action role-playing,Ion Storm


In [7]:
### Is all the games in the dataset? Why not?
len(imdb_df.loc[imdb_df.name.isin(GOTY_df.Game)].name.unique()) == len(GOTY_df.Game.unique())
print('Game Titles not in the dataset: ')
for title in GOTY_df.Game.unique():
    if title not in imdb_df.name.unique():
        print(title)

Game Titles not in the dataset: 
MediEvil 2
Pokémon Yellow
Gran Turismo 3: A-Spec
Tony Hawk's Pro Skater 2
Tom Clancy's Ghost Recon Advanced Warfighter


In [8]:
imdb_df.loc[imdb_df.name.str.startswith('Pokémon Y')]

,name,year,certificate,rating,votes,Action,Adventure,Comedy,Crime,Family,Fantasy,Mystery,Sci-Fi,Thriller
2092,Pokémon Yellow Version: Special Pikachu Edition,1998,E,8.7,"1,490",1,1,0,0,0,1,0,0,0
3487,Pokémon Y,2013,E,8.0,"1,063",1,1,0,0,1,0,0,0,0
10410,Pokémon Yellow Version: Special Pikachu Edition,1998,E,8.7,"1,491",1,1,0,0,0,1,0,0,0
10820,Pokémon Y,2013,E,7.9,"1,064",1,1,0,0,1,0,0,0,0


In [9]:
print('There are {} duplicates in the dataset.'.format(imdb_df.duplicated().sum()))
imdb_df.drop_duplicates(inplace= True)
imdb_df.info()

There are 5 duplicates in the dataset.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16153 entries, 0 to 20802
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         16153 non-null  object 
 1   year         16153 non-null  int64  
 2   certificate  6995 non-null   object 
 3   rating       9153 non-null   float64
 4   votes        9153 non-null   object 
 5   Action       16153 non-null  int64  
 6   Adventure    16153 non-null  int64  
 7   Comedy       16153 non-null  int64  
 8   Crime        16153 non-null  int64  
 9   Family       16153 non-null  int64  
 10  Fantasy      16153 non-null  int64  
 11  Mystery      16153 non-null  int64  
 12  Sci-Fi       16153 non-null  int64  
 13  Thriller     16153 non-null  int64  
dtypes: float64(1), int64(10), object(3)
memory usage: 1.8+ MB


In [10]:
print('There are duplicate game titles in the dataset {}'. format(imdb_df.duplicated(subset =['name', 'year', 'rating']).sum()))
### Remove the same year, rating and name due to system differences
imdb_df.drop_duplicates(subset = ['name', 'year', 'rating'], inplace= True)

There are duplicate game titles in the dataset 519


In [11]:
imdb_df[imdb_df.name == 'Aladdin']

,name,year,certificate,rating,votes,Action,Adventure,Comedy,Crime,Family,Fantasy,Mystery,Sci-Fi,Thriller


In [12]:
rating_sums = []
dup_titles = imdb_df.loc[imdb_df.duplicated(['name', 'year'])].name.unique()
for title in dup_titles:
    rating_sums.append(round(imdb_df.loc[imdb_df.name == title]['rating'].mean(), 2))
    
masking = dict(zip(dup_titles,rating_sums))
for dup in masking: 
    imdb_df.rating.mask(imdb_df.name == dup, masking[dup], inplace = True)
    
imdb_df.drop_duplicates(subset = ['name', 'year'], inplace= True)

In [13]:
year_dif_dups = imdb_df.loc[imdb_df.duplicated(['name'], False)].sort_values('name').name.unique()
imdb_df.drop(index = imdb_df.loc[imdb_df['name'].isin(year_dif_dups) & imdb_df.rating.isna()].index, inplace= True)

In [14]:
imdb_df.loc[imdb_df.duplicated(['name'], False)].sort_values('name')

,name,year,certificate,rating,votes,Action,Adventure,Comedy,Crime,Family,Fantasy,Mystery,Sci-Fi,Thriller
299,Aliens vs. Predator,2010,M,7.1,"1,402",1,0,0,0,0,0,0,1,0
857,Aliens vs. Predator,1999,M,8.0,626,1,0,0,0,0,0,0,1,0
3745,Area 51,1996,M,6.2,99,1,0,0,0,0,0,0,1,0
990,Area 51,2005,M,7.1,519,1,0,0,0,0,0,1,0,0
5171,Army Men,1999,T,6.2,49,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986,Xena: Warrior Princess,1999,T,6.8,166,1,1,0,0,0,1,0,0,0
3738,Xena: Warrior Princess,2006,NaN,7.6,22,1,0,0,0,0,1,0,0,0
6454,Xena: Warrior Princess,2001,E,6.4,8,1,1,0,0,0,1,0,0,0
4133,Zoo Tycoon,2001,NaN,7.4,266,1,0,0,0,0,0,0,0,0


In [15]:
for title in imdb_df[imdb_df.rating.isna()].name.unique():
    if(len(imdb_df.loc[imdb_df['name'] == title]) > 1):
        print(title)